In [1]:
!pip install -q langchain_community

In [2]:
!pip install -q unstructured

In [3]:
!pip install -q "unstructured[pdf]"

In [4]:
!sudo apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [5]:
!sudo apt-get install -q poppler-utils

Reading package lists...
Building dependency tree...
Reading state information...
poppler-utils is already the newest version (22.02.0-2ubuntu0.12).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [6]:
!sudo apt-get install -q libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

Reading package lists...
Building dependency tree...
Reading state information...
libleptonica-dev is already the newest version (1.82.0-3build1).
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-script-latn is already the newest version (1:4.00~git30-7274cfa-1.1).
python3-pil is already the newest version (9.0.1-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [7]:
!pip install -q unstructured-pytesseract
!pip install -q tesseract-ocr

In [8]:
!pip install -q "unstructured[pptx]"

In [9]:
!pip install -q "unstructured[docx]"

In [10]:
!pip install -q langchain_astradb

In [11]:
!pip install -q langchain langchain-openai datasets pypdf

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [14]:
from langchain_community.document_loaders import DirectoryLoader

In [15]:
loader=DirectoryLoader('/content/Docs')

In [16]:
splitter= RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=64)

In [17]:
docs=loader.load_and_split(text_splitter=splitter)

In [18]:
len(docs)

409

In [19]:
import os
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [22]:
import os
from google.colab import userdata
OPENAI_API_KEY=userdata.get("OPENAI")
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [23]:
#embedding=OpenAIEmbeddings()

In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/tmp/ipython-input-1874422665.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
from langchain_astradb import AstraDBVectorStore
#from langchain.indexes import VectorstoreIndexCreater

In [26]:
ASTRA_DB_ENDPOINT="https://xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN="AstraCS:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
ASTRA_DB_KEYSPACE="keyspace_name"

In [30]:
vstore=AstraDBVectorStore(
    embedding=embedding,
    collection_name="multidoc_vector",
    api_endpoint=ASTRA_DB_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [31]:
inserted_ids=vstore.add_documents(docs)

In [32]:
print(len(inserted_ids))

409


In [35]:
prompt_template="""
You are an AI assistant, answering questions from the provided context about the finetuning and rag. And the queries about the unsupervised leaning from the provided context which is not related to the rag and finetuning. Your responses should be concise and strictly related to the provided context.

CONTEXT:
{context}

QUESTION:{question}

YOUR ANSWER:
"""

In [36]:
prompt_tamplate=ChatPromptTemplate.from_template(prompt_template)

In [42]:
prompt_template = ChatPromptTemplate.from_template("""
Answer the question based only on the following context:
{context}

Question: {question}
""")

In [43]:
retriever=vstore.as_retriever(search_kwargs={"k":3})

In [44]:
retriever

VectorStoreRetriever(tags=['AstraDBVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x79e06d200e60>, search_kwargs={'k': 3})

In [39]:
!pip install -q langchain-huggingface

In [48]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

llm_endpoint = HuggingFaceEndpoint(
    repo_id="google/gemma-2-9b-it",#HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    #huggingfacehub_api_token="hf_..." # Your Free HF Token
)
llm=ChatHuggingFace(llm=llm_endpoint)

In [49]:
chain=(
    {"context":retriever,"question":RunnablePassthrough()}
    |prompt_template
    |llm
    |StrOutputParser()
)

In [51]:
response=chain.invoke("can you tell me the diffenrences between the finetuning a model and  a rag application?")

In [52]:
print(response)

Here's a breakdown of the differences between fine-tuning and using a RAG (Retrieval Augmented Generation) system, based on the provided text:

**Fine-tuning:**

* **Process:**  Altering the weights of an existing large language model (LLM) by training it on a specific dataset.
* **Goal:**  To specialize the model for a particular task or domain.
* **Pros:**
    * Can lead to highly accurate results for the targeted task.
    * Outputs are generally concise and focused.
* **Cons:**
    * Requires a substantial amount of labeled data for training.
    * Can be computationally expensive.

**RAG (Retrieval Augmented Generation):**

* **Process:**
    * Uses an external knowledge base (or database) to retrieve relevant information when responding to a query.
    * Combines the retrieved information with the LLM's generation capabilities to produce a more comprehensive and accurate answer.
* **Goal:** To leverage external knowledge to enhance the LLM's responses, especially in situations wh